# obt_soccer_matches

Esse notebook é responsável por criar uma tabela com as informações descritivas e fatos de partidas de todos os campeonatos disputados por clubes brasileiros nos últimos anos.

## Configurações Iniciais

### Importando as Bibliotecas


In [0]:
from pyspark.sql import functions as F

### Carregando os Dados

In [0]:
df_obt_brasileirao_matches = (
    spark.read.table("ifrs_soccer.gold.obt_brasileirao_matches")
    .withColumn(
        "round",
        F.col("round").cast("string")
    )
)

df_obt_copa_do_brasil_matches = (
    spark.read.table("ifrs_soccer.gold.obt_copa_do_brasil_matches")
    .withColumn(
        "round",
        F.col("round").cast("string")
    )
)

df_obt_libertadores_matches = (
    spark.read.table("ifrs_soccer.gold.obt_libertadores_matches")
)

## Tratamento dos Dados

In [0]:
df_final = (
    df_obt_brasileirao_matches
    .unionByName(df_obt_copa_do_brasil_matches)
    .unionByName(df_obt_libertadores_matches)
)

## Salvamento dos Dados

In [0]:
%sql
CREATE TABLE IF NOT EXISTS ifrs_soccer.gold.obt_soccer_matches (
    datetime TIMESTAMP COMMENT "Data e hora da partida",
    home_team STRING COMMENT "Nome do time da casa",
    away_team STRING COMMENT "Nome do time visitante",
    home_goal INT COMMENT "Gols do time da casa",
    away_goal INT COMMENT "Gols do time visitante",
    season INT COMMENT "Temporada (ano)",
    round STRING COMMENT "Rodada do campeonato",
    championship STRING COMMENT "Nome do campeonato",
    composite_key STRING PRIMARY KEY COMMENT "Chave composta única da partida formada por: home_team-away_team-season-round"
)
USING DELTA
TBLPROPERTIES ("quality"="gold", "layer"="data-engineering", "delta.enableTypeWidening"="true")
COMMENT "Tabela OBT com os resultados gerais de partidas de todos os campeonatos disputados por clubes brasileiros nos últimos anos.";

In [0]:
(
    df_final
    .write.mode("overwrite")
    .option("mergeSchema", True)
    .saveAsTable("ifrs_soccer.gold.obt_soccer_matches")
)